# **Machine Learning: Convolutional Neural Network Gesture Models**

---

#### **Ryan Harte**

## **Table of Contents**

1. [Introduction](#introduction)
2. [CNN: Project Setup](#cnn-project-setup)
3. [CNN Model: Light from Scratch](#cnn-model-light-from-scratch)
4. [CNN Model: Deep from Scratch](#cnn-model-deep-from-scratch)
5. [CNN Model: Data Augmentation](#cnn-model-data-augmentation)
6. [CNN Model: Data Augmentation Fine Tuned](#cnn-model-data-augmentation-fine-tuned)

## **Introduction**

This repository features a Jupyter Notebook that explores Convolutional Neural Networks (CNNs) in machine learning. The notebook analyzes a comprehensive dataset of images depicting human hand gestures and training a Neural Network (NN) to accurately recognize them. It is then tested on images of my own hand gestures to assess the NN ability to identify them correctly. The project also includes an evaluation of the model's accuracy and performance.

## **CNN: Project Setup**

In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import RandomZoom, RandomRotation, RandomFlip, Rescaling, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import pandas as pd
import cv2
import logging
import warnings

# Suppress warnings from the logging module
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings("ignore", category=UserWarning)


In [2]:
# Tensorflow Version
print(tf.__version__)

2.10.0


In [3]:
# Check if any GPU devices are detected
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs detected: {len(gpus)}")
else:
    print("No GPU detected.")

GPUs detected: 1


In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Set seed
SEED = 338424

# Global variables
IMG_SIZE = (64, 64)
BATCH_SIZE = 32
num_classes = 18 # Number of folders in dataset
AUTOTUNE = tf.data.AUTOTUNE

#### **Dataset: Loading, Splitting and Shuffling the Data**

In [5]:
# Load Dataset
dataset_dir = 'dataset/hagridset'
full_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    shuffle=True,
    seed=SEED,
    image_size=(IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# Split into training, validation, and test sets
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Total length of the dataset
total_size = len(full_ds)

# Compute indices for the splits
train_size = int(total_size * train_ratio)
val_size = int(total_size * val_ratio)
test_size = total_size - (train_size + val_size)

# Split the dataset and shuffle
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED)
test_ds = full_ds.skip(train_size + val_size).shuffle(test_size, seed=SEED)

# Cache the dataset in memory (or use a directory to store it on disk if necessary)
train_ds = full_ds.take(train_size).shuffle(train_size, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = full_ds.skip(train_size).take(val_size).shuffle(val_size, seed=SEED).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = full_ds.skip(train_size + val_size).cache().prefetch(buffer_size=AUTOTUNE)

# Count samples in each subset
def count_samples(dataset):
    sample_count = sum(1 for _ in dataset.unbatch())
    return sample_count

# Output the number of samples for each dataset
print(f'Using {count_samples(train_ds)} samples in the Training set')
print(f'Using {count_samples(val_ds)} samples in the Validation set')
print(f'Using {count_samples(test_ds)} samples in the Test set')

Found 125912 files belonging to 18 classes.
Using 88128 samples in the Training set
Using 25184 samples in the Validation set
Using 12600 samples in the Test set


In [6]:
# Get class names
class_names = full_ds.class_names
class_names

['call',
 'dislike',
 'fist',
 'four',
 'like',
 'mute',
 'ok',
 'one',
 'palm',
 'peace',
 'peace_inverted',
 'rock',
 'stop',
 'stop_inverted',
 'three',
 'three2',
 'two_up',
 'two_up_inverted']

## **CNN Model: Light from Scratch**

In [7]:
# Define the Light CNN Model from Scratch
def build_scratch_cnn_light():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))  # Reduced filter size
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))  # Smaller second layer
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))  # Smaller third layer
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))  # Reduced fully connected layer
    model.add(layers.Dropout(0.3))  # Reduced dropout

    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(
                  optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the lighter model
scratch_model_light = build_scratch_cnn_light()
scratch_model_light.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 16)       64        
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       1

In [8]:
# Train Light CNN Model
history_custom = scratch_model_light.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 21s 7ms/step - loss: 2.8933 - accuracy: 0.0570 - val_loss: 2.8903 - val_accuracy: 0.0590
Epoch 2/10
2754/2754 [==============================] - 17s 6ms/step - loss: 2.8910 - accuracy: 0.0572 - val_loss: 2.8903 - val_accuracy: 0.0590
Epoch 3/10
2754/2754 [==============================] - 17s 6ms/step - loss: 2.8904 - accuracy: 0.0572 - val_loss: 2.8903 - val_accuracy: 0.0590
Epoch 4/10
2754/2754 [==============================] - 17s 6ms/step - loss: 2.8904 - accuracy: 0.0572 - val_loss: 2.8903 - val_accuracy: 0.0590


### **CNN Model: Light from Scratch Evaluation**

In [9]:
# Evaluate the Light CNN Model
scratch_model_light.evaluate(test_ds)

394/394 [==============================] - 1s 3ms/step - loss: 2.8904 - accuracy: 0.0562


[2.8903844356536865, 0.056190475821495056]

#### CNN Light Model: Save and Load Model

In [10]:
# Save the Deep CNN Model
scratch_model_light.save('scratch_model_light.h5')

In [11]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_light = load_model('scratch_model_light.h5')

### **CNN Model: Light from Scratch Testing**

In [12]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_light, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 149ms/step
Prediction for myImages/three2.png: two_up
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/rock.png: two_up
1/1 [==============================] - 0s 17ms/step
Prediction for myImages/ok.jpg: two_up
1/1 [==============================] - 0s 18ms/step
Prediction for myImages/two_up_inverted.png: two_up


## **CNN Model: Deep from Scratch**

In [13]:
# Define the Deep CNN Model from Scratch
def build_scratch_cnn_deep():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5)) 
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(
                  optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the model
scratch_model_deep = build_scratch_cnn_deep()
scratch_model_deep.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_3 (Batc  (None, 64, 64, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 32)      

In [14]:
# Train Deep CNN Model
history_custom = scratch_model_deep.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 28s 10ms/step - loss: 2.8192 - accuracy: 0.1452 - val_loss: 2.4518 - val_accuracy: 0.2472
Epoch 2/10
2754/2754 [==============================] - 27s 10ms/step - loss: 1.8776 - accuracy: 0.4605 - val_loss: 1.5553 - val_accuracy: 0.5991
Epoch 3/10
2754/2754 [==============================] - 27s 10ms/step - loss: 1.3493 - accuracy: 0.6655 - val_loss: 1.0791 - val_accuracy: 0.7606
Epoch 4/10
2754/2754 [==============================] - 26s 10ms/step - loss: 1.1407 - accuracy: 0.7368 - val_loss: 1.0912 - val_accuracy: 0.7525
Epoch 5/10
2754/2754 [==============================] - 26s 10ms/step - loss: 1.0291 - accuracy: 0.7752 - val_loss: 0.9563 - val_accuracy: 0.7929
Epoch 6/10
2754/2754 [==============================] - 26s 10ms/step - loss: 0.9571 - accuracy: 0.7972 - val_loss: 0.9157 - val_accuracy: 0.8065
Epoch 7/10
2754/2754 [==============================] - 26s 10ms/step - loss: 0.9062 - accuracy: 0.8116 - val_loss: 0.8730 -

### **CNN Model: Deep from Scratch Evaluation**

In [15]:
# Evaluate the Deep CNN Model
scratch_model_deep.evaluate(test_ds)

  1/394 [..............................] - ETA: 7s - loss: 0.9141 - accuracy: 0.8125

394/394 [==============================] - 2s 4ms/step - loss: 0.8659 - accuracy: 0.8226


[0.8658940196037292, 0.8226190209388733]

#### CNN Deep Model: Save and Load Model

In [16]:
# Save the Deep CNN Model
scratch_model_deep.save('scratch_model_deep.h5')

In [17]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_deep = load_model('scratch_model_deep.h5')

### **CNN Model: Deep from Scratch Testing**

In [18]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_deep, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 171ms/step
Prediction for myImages/three2.png: three2
1/1 [==============================] - 0s 20ms/step
Prediction for myImages/rock.png: rock
1/1 [==============================] - 0s 16ms/step
Prediction for myImages/ok.jpg: call
1/1 [==============================] - 0s 17ms/step
Prediction for myImages/two_up_inverted.png: two_up_inverted


## **CNN Model: Data Augmentation**

In [19]:
# Data Augmentation
data_augmentation_layers = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
])

In [20]:
# Define the Deep CNN Model from Scratch
def build_scratch_cnn_da():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(data_augmentation_layers)
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu',
                            kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    model.add(layers.Dropout(0.3))
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5)) 
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(
                  optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the model
scratch_model_da = build_scratch_cnn_da()
scratch_model_da.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 64, 64, 3)         0         
                                                                 
 rescaling_2 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_9 (Conv2D)           (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_9 (Batc  (None, 64, 64, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 32, 32, 32)       

In [21]:
# Train Deep CNN Model
history_custom = scratch_model_da.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 305s 110ms/step - loss: 2.8940 - accuracy: 0.1170 - val_loss: 2.5491 - val_accuracy: 0.2025
Epoch 2/10
2754/2754 [==============================] - 307s 111ms/step - loss: 2.0872 - accuracy: 0.3771 - val_loss: 1.5178 - val_accuracy: 0.6123
Epoch 3/10
2754/2754 [==============================] - 306s 111ms/step - loss: 1.5462 - accuracy: 0.5936 - val_loss: 1.4807 - val_accuracy: 0.6397
Epoch 4/10
2754/2754 [==============================] - 302s 110ms/step - loss: 1.3161 - accuracy: 0.6779 - val_loss: 1.0757 - val_accuracy: 0.7542
Epoch 5/10
2754/2754 [==============================] - 299s 109ms/step - loss: 1.1897 - accuracy: 0.7224 - val_loss: 1.0930 - val_accuracy: 0.7459
Epoch 6/10
2754/2754 [==============================] - 301s 109ms/step - loss: 1.1184 - accuracy: 0.7465 - val_loss: 1.0134 - val_accuracy: 0.7724
Epoch 7/10
2754/2754 [==============================] - 301s 109ms/step - loss: 1.0543 - accuracy: 0.7632 - val_

### **CNN Model: Data Augmentation Evaluation**

In [22]:
# Evaluate the CNN Data Augmentation Deep Model
scratch_model_da.evaluate(test_ds)

394/394 [==============================] - 2s 4ms/step - loss: 0.8310 - accuracy: 0.8278


[0.8310316801071167, 0.8277778029441833]

#### CNN Data Augmentation Model: Save and Load Model 

In [23]:
# Save the Deep CNN Model
scratch_model_da.save('scratch_model_da.h5')

In [24]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_da = load_model('scratch_model_da.h5')

### **CNN Model: Data Augmentation Testing**

In [26]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_da, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 122ms/step
Prediction for myImages/three2.png: three2
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/rock.png: rock
1/1 [==============================] - 0s 18ms/step
Prediction for myImages/ok.jpg: ok
1/1 [==============================] - 0s 19ms/step
Prediction for myImages/two_up_inverted.png: two_up_inverted


## **CNN Model: Data Augmentation Fine Tuned**

In [27]:
# Define the Deep CNN Model from Scratch
def build_scratch_model_da_adapted_cnn():
    model = models.Sequential()
    model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
    model.add(data_augmentation_layers)
    model.add(layers.Rescaling(1.0 / 255))  # Normalize pixel values
   
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
   
    model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
   
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
   
    model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
  
    model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())

    model.add(layers.Flatten())
    
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(
                  optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
    return model

# Instantiate and summarize the model
scratch_model_da_adapted = build_scratch_model_da_adapted_cnn()
scratch_model_da_adapted.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 64, 64, 3)         0         
                                                                 
 rescaling_3 (Rescaling)     (None, 64, 64, 3)         0         
                                                                 
 conv2d_15 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 batch_normalization_15 (Bat  (None, 64, 64, 32)       128       
 chNormalization)                                                
                                                                 
 max_pooling2d_15 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 32, 32, 32)       

In [28]:
# Train Deep CNN Model
history_custom = scratch_model_da_adapted.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Epoch 1/10
2754/2754 [==============================] - 286s 103ms/step - loss: 1.7051 - accuracy: 0.4550 - val_loss: 1.1785 - val_accuracy: 0.6275
Epoch 2/10
2754/2754 [==============================] - 287s 104ms/step - loss: 0.8714 - accuracy: 0.7182 - val_loss: 1.0365 - val_accuracy: 0.6769
Epoch 3/10
2754/2754 [==============================] - 289s 105ms/step - loss: 0.6705 - accuracy: 0.7835 - val_loss: 0.5975 - val_accuracy: 0.8054
Epoch 4/10
2754/2754 [==============================] - 291s 106ms/step - loss: 0.5771 - accuracy: 0.8129 - val_loss: 0.5666 - val_accuracy: 0.8203
Epoch 5/10
2754/2754 [==============================] - 288s 105ms/step - loss: 0.5087 - accuracy: 0.8348 - val_loss: 0.5800 - val_accuracy: 0.8159
Epoch 6/10
2754/2754 [==============================] - 288s 104ms/step - loss: 0.4684 - accuracy: 0.8471 - val_loss: 0.5869 - val_accuracy: 0.8104
Epoch 7/10
2754/2754 [==============================] - 287s 104ms/step - loss: 0.4228 - accuracy: 0.8610 - val_

### **CNN Model: Data Augmentation Fine Tuned Evaluation**

In [29]:
# Evaluate the CNN Data Augmentation Deep Model
scratch_model_da_adapted.evaluate(test_ds)

394/394 [==============================] - 2s 4ms/step - loss: 0.4104 - accuracy: 0.8656


[0.4103633761405945, 0.8656349182128906]

#### CNN Data Augmentation Model Adapted: Save and Load Model

In [30]:
# Save the Deep CNN Model
scratch_model_da_adapted.save('scratch_model_da_adapted.h5')

In [31]:
# Load the model
from tensorflow.keras.models import load_model

# Load the model from the .h5 file
scratch_model_da_adapted = load_model('scratch_model_da_adapted.h5')

### **CNN Model: Data Augmentation Fine Tuned Testing**

In [32]:
# Testing own images
def predict_gesture(model, img_path, class_names):
    img = image.load_img(img_path, target_size=(64, 64))
    img_array = image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch dimension

    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

gesture_images = ['myImages/three2.png',
                  'myImages/rock.png',
                  'myImages/ok.jpg',
                  'myImages/two_up_inverted.png']

for img_path in gesture_images:
    predicted_gesture = predict_gesture(scratch_model_da_adapted, img_path, class_names)
    print(f"Prediction for {img_path}: {predicted_gesture}")

1/1 [==============================] - 0s 125ms/step
Prediction for myImages/three2.png: three2
1/1 [==============================] - 0s 17ms/step
Prediction for myImages/rock.png: rock
1/1 [==============================] - 0s 20ms/step
Prediction for myImages/ok.jpg: ok
1/1 [==============================] - 0s 22ms/step
Prediction for myImages/two_up_inverted.png: two_up_inverted


## **Testing Model in Realtime**

In [ ]:
# Assuming scratch_model_da2 and class_names are already defined elsewhere
# cap is the VideoCapture object
cap = cv2.VideoCapture(1)

# Adjust camera settings if needed
cap.set(3, 480)  # Adjust width
cap.set(4, 480)  # Adjust height

while True:
    # Read the frame from the camera
    success, img = cap.read()

    # Ensure that the frame was captured successfully before proceeding
    if not success:
        print("Error: Unable to capture video")
        break

    # Resize the image to the desired size
    imgsmall = cv2.resize(img, (128, 128))

    # Prepare the image for prediction
    image = np.array(imgsmall)
    image = image.reshape(1, 128, 128, 3)

    # Predict with your model
    ans = scratch_model_deep.predict(image)
    val = pd.DataFrame(ans, columns=class_names).idxmax(axis=1)

    # Display the prediction on the image
    cv2.putText(img, val.values[0], (0, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the webcam feed
    cv2.imshow("Webcam", img)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 18ms/step


# **CNN Greyscale**

# **Transfer Learning VGG-16**